In [92]:
from typing import Literal
from poor_man_lakehouse.config import settings
from poor_man_lakehouse.spark.builder import retrieve_current_spark_session
from pyiceberg.catalog import load_catalog
import ibis
from ibis import Table
import polars as pl


class IbisConnection:
    def __init__(self):
        self.connections = {
            "pyspark": ibis.pyspark.connect(session=retrieve_current_spark_session()),
            "polars": ibis.polars.connect(),
            "duckdb": ibis.duckdb.connect(database=":memory:", read_only=False),
        }
        self._catalog_config = settings.ICEBERG_STORAGE_OPTIONS | {
            "type": "rest",
            "uri": settings.NESSIE_PYICEBERG_SERVER_URI,
        }
        self.pyiceberg_catalog = load_catalog("nessie", **self._catalog_config)

    def get_connection(self, engine: Literal["pyspark", "polars", "duckdb"]):
        if engine not in self.connections:
            raise ValueError(f"Unsupported engine: {engine}")
        return self.connections[engine]

    def list_tables(self, engine: Literal["pyspark", "polars", "duckdb"]):
        con = self.get_connection(engine)
        return con.list_tables()

    def sql(self, engine: Literal["pyspark", "polars", "duckdb"], query: str):
        con = self.get_connection(engine)
        return con.sql(query)

    def read_table(
        self,
        database: str,
        table_name: str,
        engine: Literal["pyspark", "polars", "duckdb"],
    ) -> Table:
        """
        Read an Iceberg table in lazy mode using the specified engine.

        Args:
            database: The database/namespace name
            table_name: The table name
            engine: The engine to use for reading ("pyspark", "polars", "duckdb")

        Returns:
            Lazy Ibis table expression
        """
        con = self.get_connection(engine)

        if engine == "pyspark":
            # For PySpark, first check if table exists in the current context
            # Since we're using Nessie catalog and the table is already in the catalog
            try:
                # Try with just the table name first (since we set the current database)
                return con.table(table_name)
            except Exception as e:
                raise ValueError(
                    f"Could not read table {database}.{table_name} with PySpark: {e}"
                )

        elif engine == "polars":
            # For Polars, we need to use PyIceberg to get the table and then create Ibis table
            try:
                # Get the table from PyIceberg catalog
                iceberg_table = self.pyiceberg_catalog.load_table(
                    f"{database}.{table_name}"
                )
                lazyframe = pl.scan_iceberg(iceberg_table, reader_override="pyiceberg")
                con.create_table(f"{database}.{table_name}", lazyframe)
                return con.table(f"{database}.{table_name}")

            except Exception as e:
                raise ValueError(
                    f"Could not read table {database}.{table_name} with Polars: {e}"
                )

        elif engine == "duckdb":
            raise NotImplementedError(
                "DuckDB read_table is not implemented yet. Use sql() method instead."
            )


conn = IbisConnection()
print("Pyspark tables", conn.list_tables("pyspark"))
print("Polars tables", conn.list_tables("polars"))

Pyspark tables ['bids', 'prova']
Polars tables []


In [93]:
conn.read_table("default", "prova", "polars").execute()

,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00+00:00,AAPL,150.0,151.0,"[{'key': 'created_by', 'value': 'user1'}]"
1,2023-01-02 11:00:00+00:00,GOOGL,2800.0,2805.0,"[{'key': 'created_by', 'value': 'user2'}]"
2,2023-01-03 11:00:00+00:00,MSFT,300.0,305.0,"[{'key': 'created_by', 'value': None}]"


In [94]:
conn.read_table("default", "prova", "pyspark").execute()

/Users/graziano/GitHub/poor-man-lakehouse/.venv/lib/python3.12/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
25/09/06 19:03:55 WARN SimpleFunctionRegistry: The function unwrap_json_str replaced a previously registered function.
25/09/06 19:03:55 WARN SimpleFunctionRegistry: The function unwrap_json_int replaced a previously registered function.
25/09/06 19:03:55 WARN SimpleFunctionRegistry: The function unwrap_json_bool replaced a previously registered function.
25/09/06 19:03:55 WARN SimpleFunctionRegistry: The function unwrap_json_float replaced a previously registered function.


,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00,AAPL,150.0,151.0,{'created_by': 'user1'}
1,2023-01-02 11:00:00,GOOGL,2800.0,2805.0,{'created_by': 'user2'}
2,2023-01-03 11:00:00,MSFT,300.0,305.0,{'created_by': None}
